<span type="title">字符串</span> | <span type="update">2018-07-26</span> | <span type="version">1.0</span>
<span type="intro"><p class="card-text">本章主要讲解字符串。首先介绍了字符串使用基础：由于不可变性导致的问题。接着介绍了字符串常用的操作、字符串的格式化输出、使用正则表达式在字符串中进行查找、替换的方法。最后介绍了字符串的扫描输入：好用的 Scanner 类。</p></span>

# 字符串使用基础

## StringBuilder

字符串是一串不可变的 char 类型数组，这在 C 的时候，每个人都知道。在作为方法参数传入的时候，传入的其实是字符串的引用拷贝，但是因为其不可变，所以对其进行的操作将会返回一个新字符串对象，比如下面 `upper()` 所示的那样。这正是我们想要的。

但是，不可变也有其缺点。因为字符串本质是 Array，所以在执行字符串的 + 操作的时候，对于Java，则需要构造多个字符串对象，进行拼接，这样的效率较低。下列代码展示了字符串不可变对于正常使用和字符串的 + 操作的影响以及解决办法：StringBuilder，使用 `sb.append()` 代替 字符串 的 + 操作。

```java
public class StringBuilderDemo {
    //传入的是字符串的引用，因为其不可变性，所以返回时返回的是一个新对象
    //这很正常，也是我们想要的。但是对于字符串的拼接而言，效率就很低了
    public String upper(String s){
        return s.toUpperCase();
    }
    public String getString(){
        //多个String拼接，效率低（易发生在+运算符时）
        String s = new String();
        for (int i = 0; i < 10; i++){
            s += String.valueOf(i);
        }
        return s;
    }
    public String getString2(){
        //StingBuilder拼接，效率高
        StringBuilder s = new StringBuilder();
        for (int i = 0; i < 10; i++){
            s.append(String.valueOf(i));
        }
        return s.toString();
    }
    public String getString3(){
        //汇编显示其实还是字符串拼接
        //invokedynamic #5,  0
        String c = "Hello";
        String s =  "01" + c + "3" + "456789";
        //但是像这种： s = "12" + "hello"; 不属于字符串对象拼接
        return s;
    }
    public static void main(String[] args){
        StringBuilderDemo x = new StringBuilderDemo();
        System.out.println(x.getString());
        System.out.println(x.getString2());
        System.out.println(x.getString3());
    }
}
```

PS. 使用 `javap -c xxx.class` 查看汇编代码和注释

## toString

Java 对于 Map、List等自带的 toString 方法很好用，当调用一个对象并打印的时候，会自动调用其 toString 方法，返回一个字符串。对于 List 而言，其会遍历内部元素，对每个元素调用其子 toString 方法，然后拼接起来。

根据上面的知识，在 toString 中，如果有循环，那么最好使用 StringBuilder，避免 String 的 + 运算操作，这样可以提高效率。此外， toString 还有一个陷阱：

```java
public class StringPrintDemo {
    public static void main(String[] args){
        new Ind().toString();
    }
}
class Ind {
    //展示在调用 this 时发生的无限递归
    public String toString() {
        //return ""+this; // this会调用Ind的toString() 方法，结果就是无限递归
        return ""+super.toString(); //这样就没问题了
    }
}
```

# 字符串常用操作

和任何语言都类似，` length, isEmpty, endsWith(S), startWith(S), charAt(i), indexOf, contains(CS), equals, concat(S), join(CS/Iterable), format, matches(S), replace(S), split(S), trim(), toLower/UpperCase(), toCharArray()` 详见 Java Doc String Class

# 字符串的格式化

下列代码展示了字符串的格式化，等同于 C 的 printf。

```java
import java.util.Calendar;
import java.util.Formatter;
import java.util.GregorianCalendar;
import static com.mazhangjing.Print.print;

public class PrintFormat {
    static int answer = 42;
    static final double pi = 3.1415;
    public static void main(String[] args){
        //---------------直接调用静态的 System.out.printf/format 格式化---------------
        //和 C 相比，Java格式化字符串不能自动转换（float -> int）
        System.out.printf("Pi is %f, and the answer is %d\n",pi,answer);
        System.out.format("Pi is %.3f, and the answer is %010d\n",pi,answer);
                                           
        //Pi is 3.141500, and the answer is 42
        //Pi is 3.142, and the answer is 0000000042

        //--------------------调用 Formatter 进行格式化 -----------------------------
        //所有的格式化被送到Formatter进行处理，构造需传递一个输出对象，用 .format 进行格式化，并从输出进行输出。
        Formatter f = new Formatter(System.out);
        f.format("Hello from Corkine Ma. Pi is %f\n\n",pi);
                                           
        //Hello from Corkine Ma. Pi is 3.141500
                                           
        //--------------------Format.format 格式化说明符参数 -------------------------------                              
        //使用方法： %[argument_index＄][flags][width][.precision]conversion
        //其中width指定域最小宽度，-表示左对齐，0表示0开头填充，precision字符串指的是长度，浮点数指的是小数位数
        f.format("%-15s %-15s %-15s\n","Item","Qty","Price");
        f.format("%-15s %-15s %-15s\n","-----","-----","-----");
        f.format("%-15s %-15s %.2f\n","Banana",13,20.0);
        f.format("%-15s %-15s %.2f\n","Orange",10,30.0);
        f.format("%-15s %-15s %.2f\n","Apple",23,20.0);
        f.format("%-15s %-15s %.2f\n","Meat",40,30.0);//如果写错类型就会报错，30就报错
        //但是奇怪的是，数字向字符串转换则不会出错，而整数向浮点数转会出错，浮点数向整数转也会出错
        f.format("%s",10,"23"); //如果多写参数则不会出错
        
        /** Output: 
        Item            Qty             Price          
        -----           -----           -----          
        Banana          13              20.00
        Orange          10              30.00
        Apple           23              20.00
        Meat            40              30.00  **/

        //-------------------------Java 的格式化说明符------------------------------------
        //d 整数；e 浮点数E；f 浮点数；s 字符串；c Unicode字符；b 布尔值；x 十六进制；h 散列码十六进制
        f.format("\n%d,%e,%f,%s,%c,%b,%x,%h",20,13.1,2.0,"Hello",'c',0,32131,213121);
        //尤其注意：所有非null的布尔值，永远为true。0也是true
        
        //------------------------静态方法 String.format 返回字符串 ----------------------
        //下列代码展示了String.format() 这个静态方法返回一个字符串对象，而不像Formatter直接打印
        //%[argument_index$][flags][width][.precision]conversion
        //位置、左右对齐、宽度、精度、conversion包括两个字符，其一为t/T表示时间，其二表示详细日期
        Calendar c = new GregorianCalendar();
        print(String.format("\nMy birthday is %1$tm，%1$te。It is %1$tA",c));
    }
}
```

以上代码展示了 `printf(), format()` 和 `Formater().format()` 以及 `String.format()` 这三种常用的格式化方法。关于格式化符、格式化符参数以及字符串格式化格式的进一步信息，参考 Java Doc Formater 类。

需要注意：和 C 不同，不能发生除了到 String 之外的其余参数和格式化符之间的转换，即使 float 到 double 都不行，必须使用 (double)float_n 直接运算符或者方法进行转换。此外，boolean 值有单独的参数 b，并且所有不是 null 的东西都不是 false。

# 使用正则表达式进行字符串查找

Java 对于正则表达式的支持很好，在最新的SE10中，在String的很多操作中已经可以直接调用regex Pattern进行简单的操作了。首先需要注意的是，Java上的反斜杠+字母表示正则表达式字符类需要使用双反斜杠。对于普通反斜线，需要使用四反斜杠。

```java
import java.util.Arrays;
import java.util.regex.Matcher;
import java.util.regex.Pattern;
import static com.mazhangjing.Print.print;
import static com.mazhangjing.Print.printf;

public class Regular {
    /** 常见的正则表达式字符类
     * + 表示一个或多个表达式。比如\\d+ 一或多个数字
     * \\+ 表示转义过的加号
     * | 表示或者
     * （） 表示分组
     * ？表示一个元素
     * w 表示单词; W 表示非单词
     * s 空白符（空格、tab、换行、换页、回车）；S 表示非空白符
     * b 词的边界； B 表示非词的边界
     * [abc] 包含 = a|b|c
     * [^abc] 否定
     * [abc[hij]] 合并 a|b|c|h|i|j
     * [a-z&&[hij]] 相交
     * ^ ＄ 行开始和结束
     * */
                      
    //------------------------正则表达式在String类中的操作：matches, replace(All), split-------------------
    static String s = "Hello from HuaZhong Normal University. My name is Corkine Ma" +
            "and I love coding and playing Video games like OverWatch." + "The best Language" +
            "I think is Java, but C is also good. Python is good too!";
    public static void main(String[] args){
        //下面的示例展示了replace、match、split等String内部的正则表达式
        //如果不是只使用一次，那么创建正则表达式对象更高效
        System.out.println("-1231".matches("-?"));//减号开头，一个任意字符
        System.out.println("-1231".matches("-?\\d+"));//减号开头，一个任意字符，一串数字
        System.out.println("-1231".matches("(\\+|-)\\d+"));//加号或者减号开头，一串数字
        System.out.println(Arrays.toString(s.split("\\W+")));
        //切分这个很好用，生成的Array使用toString没用，必须使用Arrays.toString()方法
        //此外，如果找到匹配，那么将这个匹配删除，切分前、后两部分。这里找到的是空格（非字符）。
        System.out.println(s.replaceAll("HuaZhong","Central China"));
                                    
                      
        /**正则表达式量词
         * 贪婪 - 节制 - 作用
         * X? - X?? - 匹配一个或者0个
         * X* - X*? - 匹配0个或者多个
         * X+ - X+? - 匹配一个或多个
         * X{n} - X{n}? - 匹配n个
         * X{n,} - X{n,}? - 匹配至少n个
         * X{n,m} - X{n,m}? - 匹配至少n个，至多m个
         * */
                                           

        //---------------------------使用Pattern和Matcher捕获内容-----------------------------
        //基本使用方法----------
        //使用静态方法传入模式以创建一个Pattern对象，使用其matcher方法，传递内容，构造一个matcher对象
        Pattern p = Pattern.compile("\\b[a-f]\\w+");//捕获以a-f为首的单词
        Matcher m = p.matcher("Arline are eight apples and one orange while Anita hadn't and");
        while (m.find()){ //find() 内有一个指针，是Iterator对象
            System.out.printf("Find %s at %d - %d\n",m.group(),m.start(),m.end()-1);
            //Start表示开始的字符位置，end表示结束的位置+1
        }

        //按组匹配内容----------
        //0组指的是全部，1组指的是遇到的第一个括号，2组是遇到的第二个括号，3组是遇到的第三个括号
        Matcher m2 = Pattern.compile("(\\S+)\\s+((\\S+)\\s+(\\S+))").matcher("" +
                "Arline are eight apples and one orange while Anita hadn't and");
        while (m2.find()) {
            print("===m2");
            for (int i = 0; i < m2.groupCount(); i++){
                print("Group" + i + ": " + m2.group(i));
            }
        }

        //matchs, find, lookingat 的区别-------
        //match只有当输入都匹配才会成功，并且不像find，bool值不变
        //find会在任何位置匹配
        //lookingat只要输入的第一部分匹配就可成功
        Matcher m3 = Pattern.compile("(\\S+)\\s+((\\S+)\\s+(\\S+))").matcher("" +
                "Arline are eight");
        if (m3.matches()) {
            print("===m3");
            for (int i = 0; i < m3.groupCount(); i++){
                print("Group" + i + ": " + m3.group(i));
            }
        }

        //Pattern标记组合的使用-----------------
        Matcher m4 = Pattern.compile("java",
                Pattern.CASE_INSENSITIVE|Pattern.DOTALL).matcher("Hello Java\n" +
                "from Corkine Ma java\n");
        print("===m4");
        while (m4.find()) {printf("%s from %d\n",m4.group(),m4.start());}

        //split方法的使用--------------------
        //分割字符串，类似于String.split()
        print(Arrays.toString(
                Pattern.compile("\\s+").split("Hello from Corkine Ma")));
    }
}
Find are at 7 - 9
Find eight at 11 - 15
Find apples at 17 - 22
Find and at 24 - 26
Find and at 58 - 60
===m2
Group0: Arline are eight
Group1: Arline
Group2: are eight
Group3: are
===m2 ...
[此处 ... m2 内容已省略]
===m3
Group0: Arline are eight
Group1: Arline
Group2: are eight
Group3: are
===m4
Java from 6
java from 27
[Hello, from, Corkine, Ma]
```

上述代码展示了
- 正则表达式的基本语法、量词。
- 使用 String 类的 matches, replace(All), split 进行正则表达式的使用的方法。
- 使用 Pattern 和 Matcher 进行基本正则表达式的使用：find()、group()、start()、end()。
- 按组分配的用法
- find() 和 matches() 和 lookingAt() 的区别
- Pattern 的正则表达式标志介绍，比如忽略大小写、多行、Unix换行等
- Matcher 的 split 方法

# 字符串的替换

正则表达式可以使用多种方式进行替换：`replaceFirst() replaceAll()` 以及 `appendReplacement()`。对于前者，只用 String 类的正则表达式即可，而对于后者，可以进行处理后再进行替换。其需要构造一个 `StringBuffer` 对象，然后对于 Matcher 而言，当其找到匹配的时候，调用 `appendReplacement(new StringBuffer, toUpper etc method)` 将找到的这个匹配使用给的方法进行操作，然后将操作后的对象进行替换，然后将从头到现在的对象放置到 sb 中。如果完全替换全文，那么直接读取 sb 就是替换并处理后的字符串。如果替换几个，那么需要继续使用 `appendTail` 来拼接完整字符串。

此外，在这里介绍了 Matcher 的 reset 方法，这个方法可以重复使用 Matcher 对象，只是 reset 了其中要处理的字符串。

```java
import java.util.regex.Matcher;
import java.util.regex.Pattern;
import static com.mazhangjing.Print.print;
import static com.mazhangjing.Print.printf;

public class RegularReplace {
    public static void main(String[] args){
        String s = "/*!Here's a   block  of text to use as input to the regular expression matcher" +
                ". Note that we'll first extract the block of text by looking for the special " +
                "delimiters. then process the extracted block !*/";
        Matcher m = Pattern.compile("/\\*!(.*)!\\*/",Pattern.DOTALL).matcher(s);
        if (m.find()) {
            s = m.group(1);
            print("Find");
        }
        print(s);//将两个以上的空格替换为一个空格。
        s = s.replaceAll(" {2,}"," ");
        print(s);//替换一次：
        print(s.replaceFirst("[abc]","**a**"));
        //但是吧，如果需要对找到的元素进行特殊操作，relaceAll和replaceFirst就不能胜任了
                                           
                                           
        StringBuffer sb = new StringBuffer();
        Matcher m2 = Pattern.compile("[aeiou]").matcher(s);
        while (m2.find()){
            //将这些找到的元音字母进行某些操作后添加到sb中
            m2.appendReplacement(sb,m2.group().toUpperCase());
            if (sb.length() > 10) break;
        }
        print(sb); //可以看到，像一个传送带，头部被保存到了sb中
        m2.appendTail(sb); //这里是将未处理的部分放入sb中
        print(sb);//这样就得到了完全的信息

        //reset可以让我们重复使用一个mathcer，而不用重新创建一个matcher对象
        m.reset("/*!Something here!*/");
        while (m.find()) {printf(m.group(1));}

    }
}
Find
Here's a   block  of text to use as input to the regular expression matcher. Note that we'll first extract the block of text by looking for the special delimiters. then process the extracted block 
Here's a block of text to use as input to the regular expression matcher. Note that we'll first extract the block of text by looking for the special delimiters. then process the extracted block 
Here's **a** block of text to use as input to the regular expression matcher. Note that we'll first extract the block of text by looking for the special delimiters. then process the extracted block 
HErE's A blO
HErE's A blOck of text to use as input to the regular expression matcher. Note that we'll first extract the block of text by looking for the special delimiters. then process the extracted block 
Something here
```

# Scanner 扫描输入

在 Scannner 出现之前，Java 要想将一个字符串扫描输入，需要将这个字符串传递给 StringBuffer，然后将 StringBuffer 传递给 BufferReader，之后调用BR的 readLine 方法读取。其中这个 StringBuffer 是一个 Readable 的实现。

而现在，我们有了 Scanner，它可以接受任意 Readable 的实现（即有read()方法），然后它自身是一个 Iterator<E> 的实现，可以调用 next 等来读取。常用的 Readable 实现由 FileStream、InputStream、String等。简而言之，Readable 的文件、流或者字符串通过 Iterator 的 Scanner 可以被逐行或者逐字的读取。
    
Scanner 之所以放这里是因为其可以按照某种分界来分割读取文件，其中接受正则表达式。调用 `useDelimiter()` 来设置一个正则表达式的定界符，然后就可以按照这个标准来切分Readable实现，然后调用`nextXXX()`来进行读取了。

```java
import java.io.FileInputStream;
import java.io.FileReader;
import java.util.Scanner;
import static com.mazhangjing.Print.print;
import static com.mazhangjing.Print.printf;

public class ScannerDemo {
    public static void main(String[] args) throws Exception{
        FileInputStream f = new FileInputStream("C:/rookery.sql");
        //scanner可以接受任何readable（可以.read()）的东西，包括file、stream、string
        Scanner in = new Scanner(f);
        //print("Hello from cm\nLine 2 ");
        while (in.hasNext()){
            print(in.nextLine());
        }
        f.close();

        in = new Scanner("Hello\nfrom corkine");
        while (in.hasNext()){
            print(in.nextLine());
        }

        //展示了Scanner的定界符
        Scanner ss = new Scanner("20,23, 2131, 231 , 312");
        ss.useDelimiter("\\s*,\\s*");
        while (ss.hasNextInt()){
            printf("%d\n",ss.nextInt());
        } printf("%s",ss.delimiter()); //使用delimiter返回使用的定界符Pattern
    }
}
Hello
from corkine
20
23
2131
231
312
\s*,\s*

```